# Manhattan Neighborhood Battle - Sushi Restaurant
We will be using the Foursquare API and clustering to find the most suitable location for opening a sushi restaurant in Manhattan, New York. Opening a restaurant in a big city needs thorough planning and market research, and many factors go into deciding the most important aspect - the location.

By analyzing the number of sushi restaurants in Manhattan and making clusters based on that, we will be able to recommend the optimal location for a new sushi restaurant.

Data:
We will be using data on New York City neighborhoods already provided in the course, along with Foursquare data on the top 100 venues in a 500 radius in NY.


First, let's import the needed libraries:


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line when doing the FINAL run (after you've coded everything else and want to test)
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line (FINAL)
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.6.14

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Libraries imported.


Let's get the New York City neighborhood data:

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [6]:
neighborhoods_data = newyork_data['features']

In [7]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [10]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [11]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [13]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [14]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [15]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [16]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [17]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

# Using Foursquare API

In [21]:
CLIENT_ID = 'xxx ' # your Foursquare ID
CLIENT_SECRET = 'xxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT=100

Your credentails:
CLIENT_ID: xxx 
CLIENT_SECRET:xxx


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
# type your answer here

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [24]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3313, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [25]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,45,45,45,45,45,45
East Village,100,100,100,100,100,100


In [27]:
print('There are {} unique categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 335 unique categories.


In [28]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping 

In [29]:
manhattan_onehot.shape

(3313, 336)

In [30]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Gym,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping 

In [31]:
manhattan_grouped.shape

(40, 336)

# Let's just concentrate on sushi restaurants in the dataset.

In [33]:
len(manhattan_grouped[manhattan_grouped["Sushi Restaurant"] > 0])


24

In [34]:
sushi = manhattan_grouped[["Neighborhood","Sushi Restaurant"]]
sushi.head()

,Neighborhood,Sushi Restaurant
0,Battery Park City,0.01
1,Carnegie Hill,0.01
2,Central Harlem,0.00
3,Chelsea,0.01
4,Chinatown,0.00


Cluster the neighborhoods into 5 clusters.

In [35]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = sushi.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 3, 0, 3, 2, 3, 3, 3, 3], dtype=int32)

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Memorial Site,Wine Shop,Italian Restaurant,Clothing Store,Plaza,Men's Store
1,Carnegie Hill,Pizza Place,Coffee Shop,Café,Yoga Studio,Bookstore,Cosmetics Shop,Bar,French Restaurant,Spa,Japanese Restaurant
2,Central Harlem,African Restaurant,Gym / Fitness Center,Public Art,French Restaurant,Cosmetics Shop,American Restaurant,Chinese Restaurant,Seafood Restaurant,Art Gallery,Bar
3,Chelsea,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bakery,American Restaurant,Nightclub,Theater,Hotel,Seafood Restaurant,Cupcake Shop
4,Chinatown,Chinese Restaurant,American Restaurant,Cocktail Bar,Bubble Tea Shop,Dumpling Restaurant,Vietnamese Restaurant,Ice Cream Shop,Bakery,Malay Restaurant,Noodle House


In [61]:
# add clustering labels
sushi.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = sushi

manhattan_merged = manhattan_merged.join(manhattan_venues.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Cluster Labels,Neighborhood,Sushi Restaurant,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Battery Park City,0.01,40.711932,-74.016869,Battery Park City Esplanade,40.711622,-74.017907,Park
0,0,Battery Park City,0.01,40.711932,-74.016869,Equinox Brookfield Place,40.712704,-74.014995,Gym
0,0,Battery Park City,0.01,40.711932,-74.016869,Hudson Eats,40.712803,-74.016102,Food Court
0,0,Battery Park City,0.01,40.711932,-74.016869,"Waterfront Plaza, Brookfield Place",40.713241,-74.016241,Plaza
0,0,Battery Park City,0.01,40.711932,-74.016869,Brookfield Place (BFPL),40.713240,-74.015193,Shopping Mall


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Venue Latitude'], manhattan_merged['Venue Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine clusters

In [66]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0]


,Cluster Labels,Neighborhood,Sushi Restaurant,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Battery Park City,0.010000,40.711932,-74.016869,Battery Park City Esplanade,40.711622,-74.017907,Park
0,0,Battery Park City,0.010000,40.711932,-74.016869,Equinox Brookfield Place,40.712704,-74.014995,Gym
0,0,Battery Park City,0.010000,40.711932,-74.016869,Hudson Eats,40.712803,-74.016102,Food Court
0,0,Battery Park City,0.010000,40.711932,-74.016869,"Waterfront Plaza, Brookfield Place",40.713241,-74.016241,Plaza
0,0,Battery Park City,0.010000,40.711932,-74.016869,Brookfield Place (BFPL),40.713240,-74.015193,Shopping Mall
0,0,Battery Park City,0.010000,40.711932,-74.016869,Davidoff of Geneva,40.713111,-74.015130,Smoke Shop
0,0,Battery Park City,0.010000,40.711932,-74.016869,Mighty Quinn's BBQ,40.712542,-74.015666,BBQ Joint
0,0,Battery Park City,0.010000,40.711932,-74.016869,Le District Coffee,40.713284,-74.015854,Coffee Shop
0,0,Battery Park City,0.010000,40.711932,-74.016869,Le District,40.712878,-74.015633,Food Court
0,0,Battery Park City,0.010000,40.711932,-74.016869,Winter Garden Atrium,40.713308,-74.015842,Performing Arts Venue


In [67]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1]


,Cluster Labels,Neighborhood,Sushi Restaurant,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
16,1,Lenox Hill,0.06,40.768113,-73.958860,Whiskey & Wine Off 69,40.767272,-73.959544,Liquor Store
16,1,Lenox Hill,0.06,40.768113,-73.958860,sweetgreen,40.767128,-73.956846,Salad Place
16,1,Lenox Hill,0.06,40.768113,-73.958860,Up Thai,40.769898,-73.957598,Thai Restaurant
16,1,Lenox Hill,0.06,40.768113,-73.958860,Cigar Inn,40.768776,-73.956222,Smoke Shop
16,1,Lenox Hill,0.06,40.768113,-73.958860,La Esquina,40.769451,-73.957782,Taco Place
16,1,Lenox Hill,0.06,40.768113,-73.958860,Anthropologie,40.769296,-73.961085,Women's Store
16,1,Lenox Hill,0.06,40.768113,-73.958860,A Matter of Health,40.768028,-73.955933,Health Food Store
16,1,Lenox Hill,0.06,40.768113,-73.958860,Donna Margherita,40.766452,-73.959905,Pizza Place
16,1,Lenox Hill,0.06,40.768113,-73.958860,Bohemian Spirit Restaurant,40.769114,-73.956539,Czech Restaurant
16,1,Lenox Hill,0.06,40.768113,-73.958860,Refine Method,40.767315,-73.956887,Gym


In [68]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2]


,Cluster Labels,Neighborhood,Sushi Restaurant,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,2,Civic Center,0.020000,40.715229,-74.005415,Atera,40.716752,-74.005712,Molecular Gastronomy Restaurant
5,2,Civic Center,0.020000,40.715229,-74.005415,Nish Nūsh,40.715537,-74.007725,Falafel Restaurant
5,2,Civic Center,0.020000,40.715229,-74.005415,Gibney Dance Center Downtown,40.713923,-74.005661,Dance Studio
5,2,Civic Center,0.020000,40.715229,-74.005415,Arcade Bakery,40.717072,-74.006605,Bakery
5,2,Civic Center,0.020000,40.715229,-74.005415,Heyday,40.715598,-74.007882,Spa
5,2,Civic Center,0.020000,40.715229,-74.005415,African Burial Ground National Monument,40.714990,-74.005530,Monument / Landmark
5,2,Civic Center,0.020000,40.715229,-74.005415,Exceed Physical Culture,40.715629,-74.007992,Gym / Fitness Center
5,2,Civic Center,0.020000,40.715229,-74.005415,Racines,40.714754,-74.007581,French Restaurant
5,2,Civic Center,0.020000,40.715229,-74.005415,Sophie's Cuban Cuisine,40.714803,-74.007656,Cuban Restaurant
5,2,Civic Center,0.020000,40.715229,-74.005415,Woops!,40.716398,-74.005032,Bakery


In [69]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3]


,Cluster Labels,Neighborhood,Sushi Restaurant,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,3,Central Harlem,0.0,40.815976,-73.943211,Harlem Cycle,40.817201,-73.942592,Cycle Studio
2,3,Central Harlem,0.0,40.815976,-73.943211,Shrine World Music Venue,40.814403,-73.944242,Music Venue
2,3,Central Harlem,0.0,40.815976,-73.943211,Harlem Hops,40.814336,-73.944494,Beer Bar
2,3,Central Harlem,0.0,40.815976,-73.943211,Belle,40.816890,-73.942680,American Restaurant
2,3,Central Harlem,0.0,40.815976,-73.943211,Yatenga,40.814306,-73.944430,French Restaurant
2,3,Central Harlem,0.0,40.815976,-73.943211,NYPL - Schomburg Center for Research in Black ...,40.814384,-73.940747,Library
2,3,Central Harlem,0.0,40.815976,-73.943211,Abyssinia Restaurant,40.816308,-73.946054,Ethiopian Restaurant
2,3,Central Harlem,0.0,40.815976,-73.943211,Ponty Bistro Harlem,40.817886,-73.941522,African Restaurant
2,3,Central Harlem,0.0,40.815976,-73.943211,Harlem Nights,40.817198,-73.942146,Bar
2,3,Central Harlem,0.0,40.815976,-73.943211,Bill's Place,40.813366,-73.943701,Jazz Club


In [70]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4]

,Cluster Labels,Neighborhood,Sushi Restaurant,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
12,4,Greenwich Village,0.040000,40.726933,-73.999914,Chobani,40.725819,-74.000985,Snack Place
12,4,Greenwich Village,0.040000,40.726933,-73.999914,Banter,40.727959,-74.001205,Café
12,4,Greenwich Village,0.040000,40.726933,-73.999914,DomoDomo,40.728007,-74.001683,Sushi Restaurant
12,4,Greenwich Village,0.040000,40.726933,-73.999914,Carbone,40.727903,-74.000136,Italian Restaurant
12,4,Greenwich Village,0.040000,40.726933,-73.999914,Terra Blues,40.728288,-73.999381,Jazz Club
12,4,Greenwich Village,0.040000,40.726933,-73.999914,La Colombe Torrefaction,40.725820,-74.001192,Coffee Shop
12,4,Greenwich Village,0.040000,40.726933,-73.999914,Local,40.727089,-74.001912,Beer Bar
12,4,Greenwich Village,0.040000,40.726933,-73.999914,Miss Lily's & Melvin's Juice Box,40.727916,-74.001423,Caribbean Restaurant
12,4,Greenwich Village,0.040000,40.726933,-73.999914,Sacred Sounds Yoga,40.728638,-74.000115,Yoga Studio
12,4,Greenwich Village,0.040000,40.726933,-73.999914,Raffetto's,40.728010,-74.001949,Gourmet Shop


# Conclusion

Cluster 3 has the least sushi restaurants, and cluster 1 has the most. If an entrepreneur wants to open a sushi place where there is not much competition, he should choose to open it in one of the neighborhoods belonging to Cluster 3, which could be Soho, for example, where many high-end restaurants exists, but also places that attract tourists such as shops, art galleries and museums.

Of course, the results of this analysis are limited because only factor was taken into account (the number of sushi restaurants). Other factors, such as the crime rate, population, average income etc. could make for a better analysis.